In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.legacy.data import Field, TabularDataset, BucketIterator

from dataset.mtgcards import RuleText
from utils.preprocess import fields_for_rule_text

import random
from tqdm import tqdm

import models.model6 as model6

d:\Miniconda3\envs\seq2seq\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SRC, TRG = fields_for_rule_text(include_lengths=False, batch_first=True)
fields = {'src': ('src', SRC), 'trg': ('trg', TRG)}

train_data, valid_data, test_data = RuleText.splits(fields=fields, version='v2.2')

In [3]:
SRC.build_vocab(train_data, min_freq = 4)
TRG.build_vocab(train_data, min_freq = 4)
print(f"Unique tokens in source (en) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (zh) vocabulary: {len(TRG.vocab)}")

for x in random.sample(list(train_data), 1):
    print(x.src, x.trg)

Unique tokens in source (en) vocabulary: 1294
Unique tokens in target (zh) vocabulary: 1949
['attacking', 'does', "n't", 'cause', 'it', 'to', 'tap', '.'] ['它', '攻击', '时', '不', '需', '横置', '。']


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    sort_within_batch = True,
    sort_key = lambda x: len(x.src),
    device = device)

print(next(iter(train_iterator)))

cpu

[torchtext.legacy.data.batch.Batch of size 128]
	[.src]:[torch.LongTensor of size 128x7]
	[.trg]:[torch.LongTensor of size 128x11]


In [9]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

model = model6.create_model(INPUT_DIM, OUTPUT_DIM, SRC_PAD_IDX, TRG_PAD_IDX, device,
                            config='wide')

Load parameters from d:\Desktop\mtg-cards-translation\models\model6/configs/wide.json
Parameters: {'HID_DIM': 512, 'ENC_LAYERS': 3, 'DEC_LAYERS': 3, 'ENC_HEADS': 8, 'DEC_HEADS': 8, 'ENC_PF_DIM': 1024, 'DEC_PF_DIM': 1024, 'ENC_DROPOUT': 0.1, 'DEC_DROPOUT': 0.1}


In [10]:
from utils import count_parameters, train_loop
from models.model6.train import initialize_weights, train, evaluate
model.apply(initialize_weights)
count_parameters(model)

18534301

In [7]:
LEARNING_RATE = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)
train_loop(model, optimizer, criterion, train, evaluate,
           train_iterator, valid_iterator, 
           save_path='result/', file_name='model6-rule-v2.2.pt', load_before_train=True)

model will be saved to result/model6-rule-v2.2.pt


100%|██████████| 454/454 [00:19<00:00, 23.19it/s]


Epoch: 01 | Time: 0m 19s
	Train Loss: 2.062 | Train PPL:   7.861
	 Val. Loss: 0.686 |  Val. PPL:   1.985


100%|██████████| 454/454 [00:19<00:00, 23.53it/s]


Epoch: 02 | Time: 0m 19s
	Train Loss: 0.566 | Train PPL:   1.761
	 Val. Loss: 0.335 |  Val. PPL:   1.398


...

Epoch: 08 | Time: 0m 19s
	Train Loss: 0.142 | Train PPL:   1.152
	 Val. Loss: 0.137 |  Val. PPL:   1.147


100%|██████████| 454/454 [00:19<00:00, 23.30it/s]


Epoch: 09 | Time: 0m 19s
	Train Loss: 0.128 | Train PPL:   1.137
	 Val. Loss: 0.136 |  Val. PPL:   1.145


100%|██████████| 454/454 [00:19<00:00, 23.22it/s]


Epoch: 10 | Time: 0m 19s
	Train Loss: 0.120 | Train PPL:   1.128
	 Val. Loss: 0.128 |  Val. PPL:   1.136


In [14]:
from utils.translate import Translator
from models.model6.definition import beam_search
model.load_state_dict(torch.load('result/model6-rule-v2.2.pt', map_location=torch.device(device)))
T = Translator(SRC, TRG, model, device, beam_search)
torch.save(T,'result/model6-T-v2.2.pt')

In [15]:
data = 'Whenever <1> becomes attached to a creature, for as long as <1> remains attached to it, you may have that creature become a copy of another target creature you control.'
data = 'target creature gets - 1 / - 1 until end of turn .'
ret, prob = T.translate(data, max_len=100)
print(*ret[:3], sep='\n')

['目标', '生物', '得', '-', '1', '/', '-', '1', '直到', '回合', '结束', '。', '<eos>']
['目标', '生物', '得', '-', '1', '1', '/', '-', '1', '直到', '回合', '结束', '。', '<eos>']
['直到', '回合', '结束', '，', '目标', '生物', '得', '-', '1', '/', '-', '1', '。', '<eos>']


In [16]:
from dataset.mtgcards import TestSets
from utils import calculate_bleu
from torchtext.legacy.data import Field
from models.card_name_detector.definition import TrainedDetector
from utils.translate import sentencize, CardTranslator, CTHelper

fields = {'src-rule': ('src', Field(tokenize=lambda x: x.split(' '))), 'trg-rule': ('trg', Field())}
test_data = TestSets.load(fields)

D = TrainedDetector()

path: d:\Desktop\mtg-cards-translation\models\card_name_detector


In [19]:
dic = {}
dic = {'oil':'烁油', 'rebel':'反抗军','compleat':'完化'}
helper = CTHelper(D, dic)
silent = True
CT = CardTranslator(sentencize, T, 
                    preprocess=lambda x: helper.preprocess(x, silent), 
                    postprocess=lambda x: helper.postprocess(x, silent))

example = list(test_data)[13]
example = list(test_data)[8]
# ret = CT.translate(' '.join(example.src))
# print(ret)
for example in random.sample(list(test_data), 3):
    print(vars(example))
    ret = CT.translate(' '.join(example.src))
    print(ret)

{'src': ['Look', 'at', 'the', 'top', 'three', 'cards', 'of', 'your', 'library.', 'Put', 'one', 'of', 'them', 'into', 'your', 'hand', 'and', 'the', 'rest', 'on', 'the', 'bottom', 'of', 'your', 'library', 'in', 'any', 'order.', 'Proliferate.', '(Choose', 'any', 'number', 'of', 'permanents', 'and/or', 'players,', 'then', 'give', 'each', 'another', 'counter', 'of', 'each', 'kind', 'already', 'there.)'], 'trg': ['检视你牌库顶的三张牌。将其中一张置于你手上，其余则以任意顺序置于你的牌库底。增殖。（选择任意数量的永久物和／或牌手，然后为其已有之每种指示物各多放置一个同类的指示物。）']}
检视你牌库顶的三张牌。 将其中一张置于你手上，其余的牌则以任意顺序置于你的牌库底。 增殖。 你选择任意数量其上有指示物的永久物和／或牌手，然后在其上放置一个它已有之类别的指示物。
{'src': ['{T}:', 'Put', 'an', 'oil', 'counter', 'on', 'Rustvine', 'Cultivator.\n{T},', 'Remove', 'an', 'oil', 'counter', 'from', 'Rustvine', 'Cultivator:', 'Untap', 'target', 'land.'], 'trg': ['{T}：在锈蔓栽培师上放置一个烁油指示物。', '{T}，从锈蔓栽培师上移去一个烁油指示物：重置目标地。']}
{t}：在<rustvine cultivator>上放置一个烁油指示物。 {t}，从<rustvine cultivator>上移去一个指示物：重置目标地。
{'src': ['Menace\nAt', 'the', 'beginning', 'of', 'each', "player's", 'upkeep,', 

In [20]:
from utils import calculate_testset_bleu
calculate_testset_bleu(list(test_data)[:100], CT)

100%|██████████| 100/100 [01:50<00:00,  1.10s/it]


0.652931026205172